# PyGeom: Gemc Python Geometry Example

This example shows you the minimal Python commands needed to do something useful with the PyGeom package Python geometry.

If any of this is not clear, please email me: holtrop(at)jlab.org

## Setup

Note: Python is a highly interactive language and that makes it a lot of fun to work with. You can start a python shell by typing "python" at the command line, and follow along step by step. A slightly more convenient python shell is "ipython", which has command line completion incorporated. 

**Note**: If you want to install Python add-ons without modifying your system, check out "Python virtualenv". Even easier, CC at JLab has already installed a modern Python installation with all the scientific Python goodness as a package called Anaconda. This includes ipython and includes the MySQLdb plugin, plus a ton of other good things, see: https://docs.continuum.io/ and https://docs.continuum.io/anaconda/index. It's free to download to your own system.
We get the Anaconda python by adding it to our path. At Jlab, for bash:
> export PATH=/apps/anaconda/bin:${PATH}

For tcsh:
> setenv PATH /apps/anaconda/bin:${PATH}

We are also going to want to use ROOT to allow us to draw the geometries. This is not stricktly needed if you only want to create the TXT files or database that GEMC uses, but having it really will make the creation of geometries for GEMC easier. So before importing the GeometryROOT package (see below), you will want to have ROOTSYS setup by executing the "thisroot.sh" or "thisroot.tcsh" script. 
**At JLab** you want to do for bash:
> . /apps/root/5.34.21/root/bin/thisroot.sh 

And for the tcsh shell:
> source /apps/root/5.34.21/root/bin/thisroot.sh 

A final note, this tutorial is in a Jypyter notebook with a ROOT kernel. To install it on your own system, you need a very recent (6.06 or later) version or ROOT with the http component enabled. In my experience, on a Mac, you need to compile ROOT against the system Python 2.7 (not the Anaconda version.) See the JupyROOT documentation. All of it is still experimental. You start the notebook either as "root --notebook" or "jupyter notebook"

## A little bit of Python

We now start python with
> ipython

At the python command line you can ask for help by typing:

In [2]:
help

Type help() for interactive help, or help(object) for help about object.

In [3]:
import sys
help(sys.exit)

Help on built-in function exit in module sys:

exit(...)
    exit([status])
    
    Exit the interpreter by raising SystemExit(status).
    If the status is omitted or None, it defaults to zero (i.e., success).
    If the status is an integer, it will be used as the system exit status.
    If it is another kind of object, it will be printed and the system
    exit status will be one (i.e., failure).



For a buildin Python command, such as "import" or "print", you want to add quotes around the command when asking for help. There is lots of information, tutorials and help available on Python on the web. Just google. The language is very easy to learn.

### Notebooks:
More interesting still, this very notebook is interactive. You can alter any of the code lines, and then execute the line by pressing the shift+enter key. So you can play and modify this tutorial from your browser. You do have to execute the commands in order for them to work as expected, so you need to start at [1]. 

In [4]:
print("Each input line in this file shows the output right below. This line is Python.")

Each input line in this file shows the output right below. This line is Python.


## Getting the code

OK, now let us get started with the actual GEMC geometry stuff.

We need to get the code, which is stored on github. Simply clone the repository, or if you want to change the code, fork it or branch it. Look at Mauri's pages for some git introduction. The simplest is just to type:
> git clone https://github.com/mholtrop/PyGeom.git

You now have a directory called PyGeom, and inside it the Python package "PyGeom". In the near future, I will add a proper "setup.py" script, so you can install the package and make it part of your Python system. For now, we just work from the PyGeom directory. So
> cd PyGeom
> ipython

You can now follow along the tutorials. 

## A first simple geometry

We want to first import the packages into Python. This is done with the "import" buildin command. This will do:

In [5]:
import ROOT as R
from PyGeom import GeometryEngine, Geometry, GeometryROOT

Welcome to ROOTaaS 6.06/09
warning MySQLdb not found - no database functionality.


ImportError: No module named numpy

*Note that the last line returns a message that only shows up in this notebook, since this is JupyROOT. Also note, if MySQLdb is not installed for Python, initializeing the GeometryEngine will warn you that you will not be able to write to a database. Ask your sysadmin on how to install the MySQLdb module in Python.*

We now want to initialize the GeometryEngine. The name "TestGeom" is the name of the Gemc "detector", so the text files written out will have names like: <code>TestGeom__geometry_original.txt</code>.

In [ ]:
gen = GeometryEngine("TestGeom")

Let's define a simple scintillator paddle, dimensions 2x2x10cm, and positioned 20cm down stream of the target, which is at the origin: (0,0,0). We create the paddle by creating a Geometry object, and giving that object the needed parameter:

In [ ]:
paddle = Geometry(name="paddle_1",mother="root",description="Example of a paddle",
                  pos=[0, 0, 20], pos_units='cm', 
                  rot=[0, 0, 0], rot_units='rad',
                  col='339999', 
                  g4type='Box', 
                  dimensions=[2,2,10],dims_units='cm',
                  material='ScintillatorB')

**Some notes:**
   - If you forget what parameters are needed/allowed, type: help(Geometry) at the prompt, and look at the __init__() line which will show you all the options.
   - There is a bit of flexibility build into the Geometry. You can specify the position as a "vector": [x,y,z], or you can specify it as a string: " 10.*cm 15.2*cm 1000*mm ".
   - More flexibility, if you want units "cm" and "rad", then you can leave them out. If you want to mix units, then specify them as an array. So <code> pos="10.*cm 15.2*cm 1000*mm" </code> is the same as <code> pos=[10.,15.2,1000.],pos_units=['cm','cm','mm'] </code>
   - It is MUCH safer to specify a name for every argument, because then the order is not important. If you are careful about the order, and don't skip arguments, you can leave out the names (like C++ or Fortran).  


Now we add this geometry object to the GeometryEngine: 

In [ ]:
gen.add(paddle)

Let'd also add a target, at (0,0,0), made of liquid hydrogen (G4_lH2), and small flat pancake shape: a tube of 2 cm radius and 2 mm thickness. To show you a different way of entering the volume, we do the whole thing in one line:

In [ ]:
gen.add( Geometry(name="target",mother="root",description="A thin LH2 target",
                  pos="0*cm 0*cm 0*cm",
                  rot="0*deg 0*deg 0*deg",
                  g4type="Tube",
                  dimensions=" 0*cm 2*cm 2*mm 0*deg 360*deg",
                  material="G4_lH2",
                  col="CC0000"))

After you have added a number of items, you may now want to inspect what you have in your geometry. You can do this by simply printing the objects. You can print the GeometryEngine, to get a list of what is in it. To print the details of a specific Geometry, select it from the GeometryEngine. The Geometry engine is an itterator, so you can itterate on it to print everthing in detail. The detail you get is the same as would be printed in the GEMC geometry text file.

In [ ]:
print gen
print gen['paddle_1']
print gen[1]

In [ ]:
rr=GeometryROOT()
rr.debug = 1    # Set the debug level to 1

In [ ]:
rr.create_root_volume(size=[30,30,30],visible=0)

In [ ]:
rr.build_volumes(gen)

Note that some matials are not defined yet in the GeometryROOT package. I am working on adding these, please be patient. For showing the volumes, and for GEMC, these missing materials do not matter at all, they would only be important if you want to track particles through your ROOT geometry. Doing that would still need a significant amount of code...

## Drawing the geometry

You can now draw the geomerty by calling the "draw()" method of your GeometryROOT object.
If you do this is a *live* notebook (either a root --notebook, or jupyter notebook, or on binder), then the picture produced below will rotate, zoom etc, just as it would if you executed these command in a python shell.

In [1]:
rr.draw("oglx")    
# Note: the 'option' "olgx", draws the picture with OpenGL for this notebook. For Python, you may need "OGL" instead.
# If you leave the option out, the default will be used, which may be a wireframe picture.

NameError: name 'rr' is not defined

## Modifying
Since this is a interactive session for geometry, you probably want to be able to adjust one of your objects, and then look at it again. Let's shorten the paddle from 10 cm to 5 cm.

We need to manipulate the object inside the GeometryEngine object, "gen". What we do is change the length, and then look at it again.
**Note:** We can only have one active GeometryROOT object at one time right now. So we re-initialize our rr object to clear out the old geometry and load the new one. 

In [ ]:
gen['paddle_1'].dimensions[2]=5

In [ ]:
rr = GeometryROOT()
rr.create_root_volume()
rr.build_volumes(gen)

In [ ]:
rr.draw("oglx")

## Directly interacting with the ROOT objects
It is possible to directly interact with the underlying ROOT objects. This is more *expert level*, but I just want to show you some fun. Let's change the geometry of the paddle again, but now only in ROOT.
We now need to access the volume in the store of ROOT objects, in rr._volumes, which is a dictionary.

In [ ]:
import numpy as np
ndims=np.array([10,10,1],dtype='double')
rr._volumes['paddle_1'].GetShape().SetDimensions(ndims)
rr.draw("oglx")

That is it for now. In the next step of this tutorial, we will turn the interactive session of Python into a proper Python script that can be executed to create the GEMC geometry files (or database entries.)